In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
%cd /content/drive/MyDrive/AI/polus
!ls

/content/drive/MyDrive/AI/polus
 dataset       data.yaml   exp5   __MACOSX   obj.names	  zoloto_hack.ipynb
 dataset.zip   exp2	   exp6   obj.data   yolo.ipynb  'Видео конвейера.rar'


## Зависимости

In [62]:
!pip install sahi -qqqq

In [1]:
import os
import glob
from tqdm import tqdm
import shutil
import re
import random
import json

import cv2
from PIL import Image
import torch
from torchvision.utils import draw_bounding_boxes

import numpy as np
import pandas as pd
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
%matplotlib inline

from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [4]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances,load_coco_json

### Дата

In [ ]:
!wget -O dataset.zip http://77.244.221.121/dataset.zip
!unzip dataset.zip -d /content

## Check (h, w) in directory

In [89]:
h, w = set(), set()
public_paths = glob.glob('/content/dataset/public/*.jpg')
for path in public_paths:
    hi, wi, _ = cv2.imread(path).shape
    h.add(hi)
    w.add(wi)
h, w

({720}, {1280})

## Merge train and test annotations

In [65]:
import json
train = json.load(open('/content/dataset/annot_local/train_annotation.json', 'r'))
test = json.load(open('/content/dataset/annot_local/test_annotation.json', 'r'))

anno = {}
anno['licenses'] = train['licenses']
anno['info'] = train['info']
anno['categories'] = train['categories']
anno['images'] = []
anno['annotations'] = []

MAX_TRAIN_ID = max(set([x['id'] for x in train['images']]))
for x in test['images']:
    x['height'] = 720 # height in train annotations
    x['width'] = 1280 # width in train annotations
    x['id'] = x['id'] + (MAX_TRAIN_ID * 1000) # чтобы айди не пересекались
    anno['images'].append(x)

for x in train['images']: 
    anno['images'].append(x)

for x in test['annotations']:
    del x['iscrowd']
    x['image_id'] = x['image_id'] + (MAX_TRAIN_ID * 1000) # чтобы айди не пересекались
    x['id'] = x['id'] + (MAX_TRAIN_ID * 1000) # чтобы айди не пересекались
    anno['annotations'].append(x)

for x in train['annotations']:
    anno['annotations'].append(x)

json.dump(anno, open('/content/dataset/annot_local/traintest_annotation.json', 'w'))

In [ ]:
!du -sh dataset/annot_local/*

## Convert COCO to YOLO format

In [5]:
def create_yolo_anno(train_paths, valid_paths):
    coco = COCO('/content/dataset/annot_local/traintest_annotation.json')
    ids = list(sorted(coco.imgs.keys()))

    for i in tqdm(range(len(ids))):
        img_id = ids[i]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        path = coco.loadImgs(img_id)[0]['file_name']
        anno_path = path.split('.')[0] + '.txt'
        coco_annotation = coco.loadAnns(ann_ids)

        boxes = []
        num_objs = len(coco_annotation)

        try:
            for i in range(num_objs):
                xmin = coco_annotation[i]['bbox'][0]
                ymin = coco_annotation[i]['bbox'][1]
                xmax = xmin + coco_annotation[i]['bbox'][2]
                ymax = ymin + coco_annotation[i]['bbox'][3]
                boxes.append([xmin, ymin, xmax, ymax])
        except:
            pass
        
        rows = []
        for box in boxes:
            label_index = 0
            x_center = int((box[0] + box[2]) / 2)
            y_center = int((box[1] + box[3]) / 2)
            bbox_w = box[2] - box[0]
            bbox_h = box[3] - box[1]
            width = coco.loadImgs(img_id)[0]['width']
            height = coco.loadImgs(img_id)[0]['height']
            bbox_w = bbox_w / width
            bbox_h = bbox_h / height
            x_center = x_center / width
            y_center = y_center / height
            rows.append([label_index, x_center, y_center, bbox_w, bbox_h])

        txt_path = '/content/yolo_dataset/obj_train_data/' if path in train_paths else '/content/yolo_dataset/obj_valid_data/'
        txt_path += anno_path
        with open(txt_path, 'w') as f:
            for row in rows:
                f.write(' '.join(list(map(str, row))) + '\n')


    yolo_train, yolo_valid = [], []
    for path in train_paths:
        yolo_path = '/content/yolo_dataset/obj_train_data/' + path
        shutil.copy('/content/dataset/train/' + path, yolo_path)
        yolo_train.append(yolo_path)

    for path in valid_paths:
        yolo_path = '/content/yolo_dataset/obj_valid_data/' + path
        shutil.copy('/content/dataset/train/' + path, yolo_path)
        yolo_valid.append(yolo_path)

    with open('/content/yolo_dataset/train.txt', 'w') as f:
        f.write("\n".join(yolo_train))

    with open('/content/yolo_dataset/valid.txt', 'w') as f:
        f.write("\n".join(yolo_valid))
    
    shutil.copy('/content/drive/MyDrive/AI/polus/obj.data', '/content/yolo_dataset/obj.data')
    shutil.copy('/content/drive/MyDrive/AI/polus/obj.names', '/content/yolo_dataset/obj.names')

## YOLOv5

### SETUP

In [67]:
%cd /content
%%capture
!git clone https://github.com/ultralytics/yolov5
!cd yolov5 && pip install -r requirements.txt 

/content


UsageError: Line magic function `%%capture` not found.


In [6]:
! cp /content/drive/MyDrive/AI/polus/data.yaml /content/yolov5/data.yaml

import yaml
import io

with open("/content/yolov5/data.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)

data_loaded["train"] = "/content/yolo_dataset/obj_train_data/"
data_loaded["val"] = "/content/yolo_dataset/obj_valid_data/"

with io.open('/content/yolov5/data.yaml', 'w', encoding='utf8') as outfile:
    yaml.dump(data_loaded, outfile, default_flow_style=False, allow_unicode=True)

### Training

In [7]:
%cd /content/yolov5

/content/yolov5


In [8]:
from sklearn.model_selection import KFold

all_paths = np.array(os.listdir('/content/dataset/train/'))

In [ ]:
kf = KFold(n_splits=5)
for n_split in range(1, 2):
    shutil.rmtree('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/obj_train_data/')
    os.mkdir('/content/yolo_dataset/obj_valid_data/')

    for i, (train_index, test_index) in enumerate(kf.split(all_paths)):
        if n_split == i + 1:
            train_paths, valid_paths = all_paths[train_index], all_paths[test_index]
    
    create_yolo_anno(train_paths, valid_paths)
len(train_paths), len(valid_paths)

In [ ]:
!python train.py --img 1280 --batch 4 --epochs 20 --data data.yaml \
                --weights yolov5x6.pt --cache

In [ ]:
kf = KFold(n_splits=5)
for n_split in range(2, 3):
    shutil.rmtree('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/obj_train_data/')
    os.mkdir('/content/yolo_dataset/obj_valid_data/')

    for i, (train_index, test_index) in enumerate(kf.split(all_paths)):
        if n_split == i + 1:
            train_paths, valid_paths = all_paths[train_index], all_paths[test_index]
    
    create_yolo_anno(train_paths, valid_paths)
len(train_paths), len(valid_paths)

In [ ]:
!python train.py --img 1280 --batch 4 --epochs 20 --data data.yaml \
                --weights yolov5x6.pt --cache

In [ ]:
kf = KFold(n_splits=5)
for n_split in range(3, 4):
    shutil.rmtree('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/')
    os.mkdir('/content/yolo_dataset/obj_train_data/')
    os.mkdir('/content/yolo_dataset/obj_valid_data/')

    for i, (train_index, test_index) in enumerate(kf.split(all_paths)):
        if n_split == i + 1:
            train_paths, valid_paths = all_paths[train_index], all_paths[test_index]
    
    create_yolo_anno(train_paths, valid_paths)
len(train_paths), len(valid_paths)

In [ ]:
!python train.py --img 1280 --batch 4 --epochs 20 --data data.yaml \
                --weights yolov5x6.pt --cache

### Inference

In [16]:
%cd /content/yolov5

/content/yolov5


In [17]:
public_paths = sorted(glob.glob('/content/dataset/public/*.jpg'))

In [ ]:
! python detect.py --weights runs/train/exp14/weights/best.pt runs/train/exp11/weights/best.pt \
                                                            /content/drive/MyDrive/AI/polus/experiments/exp7/weights/best.pt \
                                                            /content/drive/MyDrive/AI/polus/experiments/exp8/weights/best.pt \
                                                            /content/drive/MyDrive/AI/polus/experiments/exp9/weights/best.pt \
                                                    --img 1280 \
                                                    --source /content/dataset/public/ \
                                                    --augment \
                                                    --conf-thres 0.05 \
                                                    --save-txt --save-conf 

In [72]:
def get_soliton_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

simple_solution = get_soliton_labels_df('runs/detect/exp19/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
simple_solution.shape

(6422, 7)

In [73]:
coco = Coco()
coco.add_category(CocoCategory(id=0, name='stone0'))
coco.add_category(CocoCategory(id=1, name='stone1'))

for path in tqdm(sorted(simple_solution['ImageID'].unique())):
    file_name = path
    image_id = int(re.findall(r'\d+', file_name)[0])
    coco_image = CocoImage(file_name=file_name, height=1080, width=1920, id=image_id)
    
    boxes = simple_solution[simple_solution['ImageID'] == file_name][['XMin', 'XMax', 'YMin', 'YMax']].values

    preds = []
    for result in boxes:
        xmin, xmax, ymin, ymax = result[0], result[1], result[2], result[3]
        xmin = int(xmin * 1280)
        ymin = int(ymin * 720)
        xmax = int(xmax * 1280)
        ymax = int(ymax * 720)
        preds.append([xmin, ymin, xmax, ymax])
    
    for box in preds:
        x_min = int(box[0])
        y_min = int(box[1])
        width = int(box[2] - x_min)
        height = int(box[3]- y_min)
        coco_image.add_annotation(
            CocoAnnotation(
            bbox=[x_min, y_min, width, height],
            category_id=1,
            category_name='stone1',
            image_id=image_id
            )
        )
    coco.add_image(coco_image)

save_json(data=coco.json, save_path='kramarenko.json')

100%|██████████| 150/150 [00:01<00:00, 145.33it/s]


#### Custom inference

In [98]:
yolo_paths = ['runs/train/exp7/weights/best.pt', 'runs/train/exp8/weights/best.pt', 'runs/train/exp9/weights/best.pt']
detectron_predictors = [predictor]
yolo_models = [torch.hub.load('ultralytics/yolov5', 'custom', path=path) for path in yolo_paths]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-15 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 416 layers, 139970872 parameters, 0 gradients, 207.9 GFLOPs
Adding AutoShape... 


In [14]:
!pip install ensemble-boxes -qqq

In [111]:
from ensemble_boxes import *

img = '/content/dataset/public/frame1335.jpg'

yolo_models = []
weights = [1] * (len(yolo_models) + 1)

iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

def ensemble_results(img, yolo_models, predictor):
    boxes_list, scores_list = [], []
    labels_list = []
    for model in yolo_models:
        results = model(img)
        pred = results.xyxy[0].detach().cpu()
        for i in range(len(pred)):
            pred[i][0] = pred[i][0] / 1280
            pred[i][2] = pred[i][2] / 1280

            pred[i][1] = pred[i][1] / 720
            pred[i][3] = pred[i][3] / 720
        boxes_list.append([x[:4].tolist() for x in pred])
        scores_list.append([x[4].tolist() for  x in pred])
        labels_list.append([1] * pred.shape[0])
    
    pred = predictor(cv2.imread(img))['instances']
    bboxes = pred.pred_boxes.tensor.cpu().tolist()
    for i in range(len(bboxes)):
        bboxes[i][0] = bboxes[i][0] / 1280
        bboxes[i][2] = bboxes[i][2] / 1280

        bboxes[i][1] = bboxes[i][1] / 720
        bboxes[i][3] = bboxes[i][3] / 720
    dscores = pred.scores.cpu().tolist()
    dlabels = pred.pred_classes.cpu().tolist()
    dlabels = [1] * len(dlabels)

    boxes_list.append(bboxes)
    scores_list.append(dscores)
    labels_list.append(dlabels)

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    return boxes

ensemble_results(img, yolo_models, predictor)

[[[0.32008092403411864, 0.8573137919108073, 0.43837652206420896, 0.9791687859429253], [0.37537851333618166, 0.5171982659233941, 0.4240745544433594, 0.5800060696072049], [0.37093777656555177, 0.6368573930528428, 0.46719040870666506, 0.7224170260959202], [0.3313652276992798, 0.505009036593967, 0.3773121118545532, 0.5580362108018663], [0.4850620269775391, 0.6537808312310113, 0.5707844734191895, 0.7422096252441406], [0.44458885192871095, 0.8014570448133681, 0.5391969680786133, 0.9749376085069444], [0.44597296714782714, 0.7067598554823133, 0.5251782894134521, 0.8166696336534288], [0.30883541107177737, 0.7512752109103733, 0.4024529933929443, 0.9161632113986545], [0.2959833383560181, 0.6126026577419705, 0.36596319675445554, 0.677711910671658], [0.43532285690307615, 0.884014892578125, 0.49391698837280273, 0.9972827487521702], [0.32147271633148194, 0.5523226420084636, 0.374441933631897, 0.6006682501898871], [0.5064642906188965, 0.5722915225558811, 0.5321310043334961, 0.6121859656439887], [0.360

array([[    0.32008,     0.85731,     0.43838,     0.97917],
       [    0.37538,      0.5172,     0.42407,     0.58001],
       [    0.37094,     0.63686,     0.46719,     0.72242],
       [    0.33137,     0.50501,     0.37731,     0.55804],
       [    0.48506,     0.65378,     0.57078,     0.74221],
       [    0.44459,     0.80146,      0.5392,     0.97494],
       [    0.44597,     0.70676,     0.52518,     0.81667],
       [    0.30884,     0.75128,     0.40245,     0.91616],
       [    0.29598,      0.6126,     0.36596,     0.67771],
       [    0.43532,     0.88401,     0.49392,     0.99728],
       [    0.32147,     0.55232,     0.37444,     0.60067],
       [    0.50646,     0.57229,     0.53213,     0.61219],
       [    0.36031,     0.56333,     0.41485,     0.63478]])

## Make submit

In [110]:
coco = Coco()
coco.add_category(CocoCategory(id=0, name='stone0'))
coco.add_category(CocoCategory(id=1, name='stone1'))

for path in tqdm(public_paths):
    file_name = path.split('/')[-1]
    image_id = int(re.findall(r'\d+', file_name)[0])
    coco_image = CocoImage(file_name=file_name, height=1080, width=1920, id=image_id)
    
    boxes = ensemble_results(path, yolo_models, predictor)
    preds = []
    for result in boxes:
        xmin, ymin, xmax, ymax = result[0], result[1], result[2], result[3]
        xmin = int(xmin * 1280)
        ymin = int(ymin * 720)
        xmax = int(xmax * 1280)
        ymax = int(ymax * 720)
        preds.append([xmin, ymin, xmax, ymax])

    for box in preds:
        x_min = int(box[0])
        y_min = int(box[1])
        width = int(box[2] - x_min)
        height = int(box[3]- y_min)
        coco_image.add_annotation(
            CocoAnnotation(
            bbox=[x_min, y_min, width, height],
            category_id=1,
            category_name='stone1',
            image_id=image_id
            )
        )
    coco.add_image(coco_image)

save_json(data=coco.json, save_path='kramarenko.json')

100%|██████████| 150/150 [00:26<00:00,  5.68it/s]
